# sklearn-porter

Repository: https://github.com/nok/sklearn-porter

## LinearSVC

Documentation: [sklearn.svm.LinearSVC](http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html)

### Loading data:

In [2]:
from sklearn.datasets import load_iris

iris_data = load_iris()
X, y = iris_data.data, iris_data.target

print X.shape, y.shape

(150, 4) (150,)


### Train classifier:

In [3]:
from sklearn import svm

clf = svm.LinearSVC(C=1., random_state=0)
clf.fit(X, y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0)

### Transpile classifier:

In [4]:
from sklearn_porter import Porter

output = Porter(clf, language='c').export()

print output

#include <stdlib.h>
#include <stdio.h>
#include <math.h>

int predict (float atts[4]) {

    double coefs[3][4] = {{0.18424209458473811, 0.45123000025163923, -0.80794587716737576, -0.45071660033253858}, {0.052877455748516447, -0.89214995228605254, 0.40398084459610972, -0.9376821661447452}, {-0.85070784319293802, -0.98670214922204336, 1.381010448739191, 1.8654095662423917}};
    double inters[3] = {0.10956266406702335, 1.6636707776739579, -1.7096109416521363};

    double class_val = -INFINITY;
    int class_idx = -1;
    int i, j;
    for (i = 0; i < 3; i++) {
        double prob = 0.;
        for (j = 0; j < 4; j++) {
            prob += coefs[i][j] * atts[j];
        }
        if (prob + inters[i] > class_val) {
            class_val = prob + inters[i];
            class_idx = i;
        }
    }
    return class_idx;
}

int main(int argc, const char * argv[]) {
    float atts[argc-1];
    int i;
    for (i = 1; i < argc; i++) {
        atts[i-1] = atof(argv[i]);
    }
    printf("%d"